# **Overview**

Hello, this notebook should get you searching approx 200 finance papers. Our system uses RAG to idnetify top Vectors. If you are looking to do different methods like clustering, or finding similr documents, you'll need to extract the JSON ( bottom few code blocks ) and build your own search on-top of it.

**Questions?** Reach out to cameron@tadatoday.ai and CC your processor.

## DO NOT MAKE CHANGES DIRECTLY IN THE NOTEBOOK, IF YOU WISH TO MAKE CHANGES PLEASE COPY IT.

In [ ]:
!pip install tada_AI

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


***JSON EXTRACT*** If you want to extract the JSON structure from the documents, please use the below code.


In [ ]:
# TadaAI setup
from tada_ai import TadaAIClient,ChunkOptions, RerankerOptions
api_key = '01a5b3ca97381ba914d72e95473c03647afd93a923ca1fa4d8193bc5ad8d92395022c2cba8d8c811635808e90fc31744'
client = TadaAIClient(api_key=api_key)


In [ ]:
#file ids for all files
client = TadaAIClient(api_key = api_key)
files = client.space_files.list('clyhqb7pk00d3hasunnfc9jj5').data
file_ids = [file.id for file in files]

In [ ]:
file_ids

['clyhqftrj00djhasu7m5rde2v',
 'clyhqfuod00dlhasum44x8lcz',
 'clyhqfwk600dnhasus3ukes93',
 'clyhqfyse00dphasubm9d9rli',
 'clyhqfzka00drhasus5muxd09',
 'clyhqg0j200dthasuesw3oh3s',
 'clyhqu4tm00qvhasuwmt8c6wu',
 'clyhqu6us00qxhasubmpj7zgl',
 'clyhqg1rm00dvhasugz7atg5i',
 'clyhqg2yb00dxhasujnftn9gf',
 'clyhqg42i00dzhasumav6bqjq',
 'clyhqg68y00e1hasu0la99ebm',
 'clyhqg7fm00e3hasuhmqfd575',
 'clyhqg9t300e5hasubv5wpc4l',
 'clyhqgas200e7hasub5oubnwz',
 'clyhqgdyh00e9hasuu1q1oqa0',
 'clyhqgfjp00ebhasukma2fuin',
 'clyhqgh5t00edhasuaitk9cuj',
 'clyhqgjjp00efhasurhusma1a',
 'clyhqgkyu00ehhasu0w95ezle',
 'clyhqgn4300ejhasus4sy9fpr',
 'clyhqgo4u00elhasupsdit44z',
 'clyhqgq8y00enhasuiforqgqn',
 'clyhqgr5m00ephasuu47vrolj',
 'clyhqgs5m00erhasu9co8lllm',
 'clyhqgv0500ethasuel4e77wu',
 'clyhqgvs900evhasuujl8sjbf',
 'clyhqgwj200exhasuhr5h0qh4',
 'clyhqgy6l00ezhasu9bc6nhc5',
 'clyhqgzs600f1hasutj9sbsvc',
 'clyhqh0l700f3hasuazum8bd4',
 'clyhqh1tq00f5hasu0aaofuls',
 'clyhqh2hd00f7hasu7vbu0rdz',
 'clyhqh31

In [ ]:
from tada_ai.exceptions import ApiException, BadRequestException
from openai import OpenAI
import json
import re

# Retrieve content blocks and save to a file
with open('Tada_JSON.txt', 'w') as f:
    for i, file_id in enumerate(file_ids):
        try:
            content_blocks = client.space_files.content_blocks(file_id)
            f.write(content_blocks.model_dump_json(indent=2) + "\n")
        except BadRequestException as e:
            print(f"Error for file ID {file_id}: {e}")

# Read the saved JSON data
with open('Tada_JSON.txt', 'r') as f:
    lines = f.readlines()

Error for file ID clyhqu4tm00qvhasuwmt8c6wu: (None)
Reason: None
HTTP response body: No such object: tada-production/spaces/clyhqb7pk00d3hasunnfc9jj5/files/clyhqu4tm00qvhasuwmt8c6wu/content-blocks.json

Error for file ID clyhqjh6k00hzhasu1383iyz8: (None)
Reason: None
HTTP response body: No such object: tada-production/spaces/clyhqb7pk00d3hasunnfc9jj5/files/clyhqjh6k00hzhasu1383iyz8/content-blocks.json

Error for file ID clyhqs22800qfhasu4em3vp64: (None)
Reason: None
HTTP response body: No such object: tada-production/spaces/clyhqb7pk00d3hasunnfc9jj5/files/clyhqs22800qfhasu4em3vp64/content-blocks.json

Error for file ID clyhqsc0l00qhhasunb3lpc8y: (None)
Reason: None
HTTP response body: No such object: tada-production/spaces/clyhqb7pk00d3hasunnfc9jj5/files/clyhqsc0l00qhhasunb3lpc8y/content-blocks.json

Error for file ID clyhqre0w00qbhasuica0gejo: (None)
Reason: None
HTTP response body: No such object: tada-production/spaces/clyhqb7pk00d3hasunnfc9jj5/files/clyhqre0w00qbhasuica0gejo/conten

## find each pdf's title id (root ids)

In [ ]:
import re
import json

def find_all_root_ids(json_string):
    pattern = r'(?:"id"\s*:\s*"([^"]+)"[^}]*"type"\s*:\s*"Root"|"type"\s*:\s*"Root"[^}]*"id"\s*:\s*"([^"]+)")'
    matches = re.findall(pattern, json_string, re.DOTALL)
    return [match[0] or match[1] for match in matches]

file_path = '/content/Tada_JSON.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = file.read()

root_ids = find_all_root_ids(json_data)


print(f"Found {len(root_ids)} Root IDs:")
for id in root_ids:
    print(id)

# Additional debugging: Find all "type": "Root" occurrences
root_types = re.findall(r'"type"\s*:\s*"Root"', json_data)
print(f"\nFound {len(root_types)} 'type': 'Root' occurrences")

if len(root_types) != len(root_ids):
    print("Warning: Number of 'Root' types doesn't match number of IDs found.")
    print("This might indicate missed IDs or structural issues in the JSON.")

Found 265 Root IDs:
wpkhbbq503gkv9kex2jzyy8l
lown3b1xir186qw94wt4h3bp
ugb9w2ei299yp3dmvaiqbhle
cliiuo1ifp9pxdias35obnv1
dnh41qpn9hfxk1y70jsgxvzz
z0ys25mdkwkcxkkr1vem5nkx
g8ttst02ixn1xd4plxyot6xu
uydtrdvoiguzsydadiq9jyhy
ttl65tr2i5bezjqaxu9j2qip
hy8lth4d5y7g9zt3jbtlyosu
yie9m7flco8xp9qqcnrs283d
qgf3t0hc0em49wvb2vbxopkh
z13mph15uhjwdiawq7hfcokj
dlw9dtrt5617om0qkg7m3gww
ass0gsfapl507rf4kir4i1ab
cyii3ovbxzu2gn2nslw4my3b
phy8qb3i8uynpuqr18qsazsi
zi504tfpork85pmm1m9m94yq
s10q9e6ovp77kiw51thvwlho
x2vek6609a6t02d7rrd3bklw
ypyzeathle6nqsbief9bto45
gcu2j39al6c6h79i7eyjo617
xsi5oydwftqv6e955xyqcbv3
apsuvxfchr8bu7o9hfdygg8l
rrobfyuo2a2ay26wfl7kb814
ypv1b0b5m56mglstnrmr0fnk
kv13il89awgi3586g7yvihbu
nwt75abwue05i63yr4g6cqt6
f1hg81nl1h4sdm87wuwqmg5s
oitym5pkug7efpme48tl4104
p9qzpulfnsemri287j7rg7uz
oa4cbctmwipq63y6uzr5odn3
pyaanyol8et44hcynjyvke89
izw4bmvyydokoh7kl2unr1vz
uodgm7760kx0y7p9vq027uah
qpq3c4u47dig3yrr8rpo3w4x
kwy5025gc9brh959740p231b
a02jo668nay6sco9vw9e7au1
ur17j5847hryjcqjjffvdm8c
qe1qj

## find each pdf's content (text between two root ids)

In [ ]:
def extract_text_between_ids(json_string, root_ids):
    result = {}
    for i in range(len(root_ids)):
        start_id = root_ids[i]
        end_id = root_ids[i+1] if i+1 < len(root_ids) else None

        start_pattern = f'"id"\\s*:\\s*"{start_id}"'
        end_pattern = f'"id"\\s*:\\s*"{end_id}"' if end_id else '$'

        pattern = f'{start_pattern}(.*?){end_pattern}'
        match = re.search(pattern, json_string, re.DOTALL)

        if match:
            text = match.group(1)
            # Extract all "text" fields from the matched section
            text_fields = re.findall(r'"text"\s*:\s*"((?:[^"\\]|\\.)*)"', text)
            result[start_id] = text_fields

    return result



In [ ]:
# Print results
text_between_ids = extract_text_between_ids(json_data, root_ids)

for id, text_fields in text_between_ids.items():
    print(f"ID: {id}")
    if text_fields:
        print(f"Title: {text_fields[0]}")
        full_text = " ".join(text_fields[1:])
        print(f"Full Text: {full_text[:200]}...")
    else:
        print("No text found for this ID")
    print()




ID: wpkhbbq503gkv9kex2jzyy8l
Title: Representation Learning: A Review and New Perspectives
Full Text: Yoshua Bengio†, Aaron Courville, and Pascal Vincent† Department of computer science and operations research, U. Montreal † also, Canadian Institute for Advanced Research (CIFAR) (cid:70) rpA ]GL.sc[ A...

ID: lown3b1xir186qw94wt4h3bp
Title: HIGH FREQUENCY MARKET MAKING
Full Text: RENE´ CARMONA AND KEVIN WEBSTER t cO Abstract. Since they were authorized by the U.S. Security and Exchange Commission in 1998, electronic exchanges have boomed, and by 2010 high frequency trading acc...

ID: ugb9w2ei299yp3dmvaiqbhle
Title: naJ
Full Text: ]MP.nif-q[ 1v4914.1031:viXra Acknowledgements Words wouldn’t suﬃce for the deep sense of gratitude with which I would like to sincerely thank my guide Dr. Abhijit Kulkarni for his guidance throughout ...

ID: cliiuo1ifp9pxdias35obnv1
Title: Analysis of Volatility Surfaces with Computer Vision
Full Text: Alec Janowski Matt Johnson Annette Jing Stanford Univers

## get the  AI quotes corresponding to the prompt

In [ ]:
import re
import json
from openai import OpenAI
from tada_ai import TadaAIClient, ChunkOptions, RerankerOptions
import difflib

# TadaAI setup
api_key = '01a5b3ca97381ba914d72e95473c03647afd93a923ca1fa4d8193bc5ad8d92395022c2cba8d8c811635808e90fc31744'
client = TadaAIClient(api_key=api_key)

# Retrieve all file IDs
files = client.space_files.list('clyhqb7pk00d3hasunnfc9jj5').data
file_ids = [file.id for file in files]

# Function to split file_ids into batches of 30
def batch_file_ids(file_ids, batch_size=30):
    return [file_ids[i:i + batch_size] for i in range(0, len(file_ids), batch_size)]

# Split file_ids into batches
batched_file_ids = batch_file_ids(file_ids)

# Your prompt
prompt = "What are transaction costs and how do they impact trading strategies"

# Function to perform search on a batch of file_ids
def search_batch(batch):
    return client.search(
        prompt,
        file_ids=batch,
        chunks=ChunkOptions(
            requested_min_word_count=50,
            requested_max_word_count=1500
        ),
        reranker=RerankerOptions(
            top_n=20,
            threshold=0.25
        ),
    )

# Perform search on each batch and collect results
all_results = []
for i, batch in enumerate(batched_file_ids):
    batch_result = search_batch(batch)
    all_results.extend(batch_result.chunks)
    print(f"Processed batch {i+1}/{len(batched_file_ids)}, found {len(batch_result.chunks)} relevant chunks")

print(f"Total chunks found across all batches: {len(all_results)}")

# Sort all results by score (assuming higher score is better)
all_results.sort(key=lambda x: x.score, reverse=True)

# We're no longer limiting to top 20 overall
final_results = all_results

# Create a dictionary of titles and full texts
titles_and_texts = {}
for id, text_fields in text_between_ids.items():
    if text_fields:
        titles_and_texts[id] = {
            'title': text_fields[0],
            'full_text': " ".join(text_fields[1:])
        }

# Prepare the document string
document_string = ""
for chunk in final_results:
    if chunk.id in titles_and_texts:
        document_string += f"[Title: {titles_and_texts[chunk.id]['title']}]\n{chunk.markdown}\n\n"
    else:
        document_string += f"[Title: Unknown]\n{chunk.markdown}\n\n"

document_chunks = f"""
QUESTION: {prompt}
DOCUMENTATION: {document_string}
"""

# Create an OpenAI client
openai = OpenAI(
    api_key="y5d2N6CCWcEM47fzswRSFhUqe5KUyC1W",
    base_url="https://api.deepinfra.com/v1/openai",
)
def split_chunks(text, max_length=30000):
    """Split the text into chunks of maximum length."""
    chunks = []
    current_chunk = ""
    for line in text.split('\n'):
        if len(current_chunk) + len(line) + 1 > max_length:
            chunks.append(current_chunk.strip())
            current_chunk = line
        else:
            current_chunk += '\n' + line
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def get_ai_response(prompt, document_chunks, max_retries=3):
    chunks = split_chunks(document_chunks)
    all_responses = []

    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}")
        for attempt in range(max_retries):
            try:
                chat_completion = openai.chat.completions.create(
                    model="Qwen/Qwen2-72B-Instruct",
                    messages=[{"role": "user", "content": f'''
                        You are an expert research assistant. Here is a part of a document you will analyze:
                        Find relevant quotes from this part of the document that are most relevant to answering the question about {prompt}.
                        Print the quotes in numbered order. Quotes should be relatively short. Include the title of the PDF each quote is from.
                        {chunk}
                        If there are no relevant quotes in this part, write "No relevant quotes found in this part."
                        '''}],
                    max_tokens=1000
                )

                response = chat_completion.choices[0].message.content
                all_responses.append(response)
                break
            except Exception as e:
                print(f"Error in attempt {attempt + 1} for chunk {i+1}: {str(e)}")
                if attempt == max_retries - 1:
                    all_responses.append("Failed to process this part of the document.")

    # Combine all responses
    combined_response = "\n".join(all_responses)

    # Generate final answer
    try:
        final_answer = openai.chat.completions.create(
            model="Qwen/Qwen2-72B-Instruct",
            messages=[{"role": "user", "content": f'''
                Based on the following quotes and information, answer the question: "{prompt}"
                Start your answer with "Answer:" and make it at least two paragraphs long.
                {combined_response}
                '''}],
            max_tokens=1000
        )
        return combined_response + "\n\nAnswer: " + final_answer.choices[0].message.content
    except Exception as e:
        print(f"Error generating final answer: {str(e)}")
        return combined_response + "\n\nFailed to generate a final answer."

# Get the raw response
raw_response = get_ai_response(prompt, document_chunks)

print(f"Number of chunks used in the final result: {len(final_results)}")
print(raw_response)

Processed batch 1/10, found 16 relevant chunks
Processed batch 2/10, found 18 relevant chunks
Processed batch 3/10, found 14 relevant chunks
Processed batch 4/10, found 15 relevant chunks
Processed batch 5/10, found 12 relevant chunks
Processed batch 6/10, found 12 relevant chunks
Processed batch 7/10, found 13 relevant chunks
Processed batch 8/10, found 14 relevant chunks
Processed batch 9/10, found 16 relevant chunks
Processed batch 10/10, found 11 relevant chunks
Total chunks found across all batches: 141
Processing chunk 1/16
Processing chunk 2/16
Processing chunk 3/16
Processing chunk 4/16
Processing chunk 5/16
Processing chunk 6/16
Processing chunk 7/16
Processing chunk 8/16
Processing chunk 9/16
Processing chunk 10/16
Processing chunk 11/16
Processing chunk 12/16
Processing chunk 13/16
Processing chunk 14/16
Processing chunk 15/16
Processing chunk 16/16
Number of chunks used in the final result: 141
1. "Price impact refers to price movements induced purely by trading flows, inde

### According the the AI quotes, find the top 10 similar quotes from pdf files(with pdf titles and similarity scores) and answers by AI

similarity method: TF-IDF vectorization and cosine similarity

(The process ensures that quotes are sourced from different documents and meet a minimum similarity threshold.)

In [ ]:
print(raw_response)

1. "Price impact refers to price movements induced purely by trading flows, independently of their information content. For large investors, such adverse price moves caused by their own trades are the main source of transaction costs."
  2. "The price forecast is commonly called an alpha signal. Turning the latter into optimal trades in turn requires an appropriate price impact model."
  3. "Price impact models are thus essential tools in algorithmic trading, allowing investment teams to estimate the effect of their trades on asset prices and thereby design, size and deploy systematic strategies."
  4. "We quantify how incorrect calibration of these price impact parameters leads to significant misspecification costs for traders and portfolio managers."
  5. "Concavity describes how the marginal build-up of price impact slows beyond a certain order size: larger orders are cheaper than a linear model suggests."
  6. "Impact decay describes how long trade-induced price moves linger in the

In [ ]:
import re
import difflib
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
#nltk.download('punkt')

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def find_best_sentence_match(ai_quote, full_text):
    sentences = sent_tokenize(full_text)
    preprocessed_sentences = [preprocess_text(s) for s in sentences if preprocess_text(s)]

    if not preprocessed_sentences:
        return None, 0

    if len(preprocessed_sentences) == 1:
        return preprocessed_sentences[0], difflib.SequenceMatcher(None, ai_quote, preprocessed_sentences[0]).ratio()

    try:
        vectorizer = TfidfVectorizer().fit(preprocessed_sentences + [ai_quote])
        sentence_vectors = vectorizer.transform(preprocessed_sentences)
        quote_vector = vectorizer.transform([ai_quote])

        similarities = cosine_similarity(quote_vector, sentence_vectors)[0]
        best_match_index = similarities.argmax()

        return sentences[best_match_index], similarities[best_match_index]
    except ValueError:
        best_match = max(preprocessed_sentences, key=lambda x: difflib.SequenceMatcher(None, ai_quote, x).ratio())
        return best_match, difflib.SequenceMatcher(None, ai_quote, best_match).ratio()

def find_quote_sources(ai_quotes):
    used_ids = set()
    quote_sources = []

    for ai_quote in ai_quotes:
        best_match = None
        best_ratio = 0
        for id, data in titles_and_texts.items():
            if id in used_ids:
                continue

            match_sentence, match_ratio = find_best_sentence_match(ai_quote, data['full_text'])
            if match_sentence and match_ratio > best_ratio:
                best_ratio = match_ratio
                best_match = (id, data['title'], match_sentence)

        if best_match and best_ratio > 0.3:  # Adjust threshold as needed
            id, title, matched_quote = best_match
            quote_sources.append((ai_quote, matched_quote, title, id, best_ratio))
            used_ids.add(id)
        else:
            quote_sources.append((ai_quote, None, None, None, 0))

    return quote_sources

def process_ai_response(response):
    print("Raw response length:", len(response))

    # Updated regex pattern to match the actual quote format
    quote_pattern = r'(\d+)\.\s+"([^"]+)"'
    ai_quotes = re.findall(quote_pattern, response)

    print(f"Number of quotes extracted: {len(ai_quotes)}")
    for i, (number, quote) in enumerate(ai_quotes[:5]):  # Print first 5 quotes
        print(f"Quote {number}: {quote[:100]}...")  # Print first 100 chars of each quote

    quote_sources = find_quote_sources([quote for _, quote in ai_quotes])

    processed_quotes = []
    for i, (ai_quote, matched_quote, title, id, match_ratio) in enumerate(quote_sources):
        if matched_quote:
            processed_quotes.append(f'[{i+1}] "{matched_quote}" (From: {title}, ID: {id}, Match: {match_ratio:.2f})')
        else:
            processed_quotes.append(f'[{i+1}] Unable to find a matching quote in the original documents for: "{ai_quote[:50]}..."')

    # Extract the answer from the original response
    answer_start = response.find("Answer:")
    answer = response[answer_start:] if answer_start != -1 else "No answer provided."

    return processed_quotes, answer

# Use the function
processed_quotes, answer = process_ai_response(raw_response)

print("\nNumber of processed quotes:", len(processed_quotes))
print("Processed Quotes:")
for quote in processed_quotes:
    print(quote)

print("\nFinal Answer:")
print(answer)

Raw response length: 21674
Number of quotes extracted: 54
Quote 1: Price impact refers to price movements induced purely by trading flows, independently of their infor...
Quote 2: The price forecast is commonly called an alpha signal. Turning the latter into optimal trades in tur...
Quote 3: Price impact models are thus essential tools in algorithmic trading, allowing investment teams to es...
Quote 4: We quantify how incorrect calibration of these price impact parameters leads to significant misspeci...
Quote 5: Concavity describes how the marginal build-up of price impact slows beyond a certain order size: lar...

Number of processed quotes: 54
Processed Quotes:
[1] "Introduction Price impact refers to price movements induced purely by trading flows, independently of their information content." (From: THE COST OF MISSPECIFYING PRICE IMPACT, ID: yv50hvr7mhpwufo9ekaf2fwo, Match: 0.71)
[2] Unable to find a matching quote in the original documents for: "The price forecast is commonly cal